In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Загрузим данные с гугл драйва

In [0]:
import os

In [0]:
import zipfile
with zipfile.ZipFile("/content/gdrive/My Drive/Churn_task.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/gdrive/My Drive/Churn_task/")

In [4]:
os.listdir("/content/gdrive/My Drive/Churn_task")

['__MACOSX',
 'TEST_RAW_DATA.csv',
 'test_submit_example.csv',
 'TRAIN_PREPARED.csv',
 '╨₧╨┐╨╕╤ü╨░╨╜╨╕╨╡ ╨╖╨░╨┤╨░╨╜╨╕╤Å.txt',
 'TRAIN_ADDITIONAL_DATA.csv',
 'TRAIN_RAW_DATA.csv',
 'TEST_PREPARED.csv',
 'TEST_ADDITIONAL_DATA.csv']

In [0]:
import pandas as pd
import numpy as np
import random
import datetime

In [0]:
df=pd.read_csv('/content/gdrive/My Drive/Churn_task/TRAIN_PREPARED.csv')

Посмотрим на данные 

In [7]:
df.head()

,cut_date,days_to_end,email,first_date,last_date,num_country_max_1days,num_city_max_1days,android_max_1days,smarttv_max_1days,iphone_max_1days,ipad_max_1days,apple_max_1days,pc_max_1days,time_spent_sum_3days,time_spent_max_3days,num_title_ep_sum_3days,num_title_ep_max_3days,num_title_ru_sum_3days,num_title_ru_max_3days,num_country_max_3days,num_city_max_3days,other_content_sum_3days,top_1_sum_3days,top_2_sum_3days,top_3_sum_3days,top_4_sum_3days,top_5_sum_3days,top_6_sum_3days,top_7_sum_3days,top_8_sum_3days,top_9_sum_3days,top_10_sum_3days,android_max_3days,smarttv_max_3days,iphone_max_3days,ipad_max_3days,apple_max_3days,pc_max_3days,time_spent_sum_7days,time_spent_max_7days,...,top_10_sum_21days,android_max_21days,smarttv_max_21days,iphone_max_21days,ipad_max_21days,apple_max_21days,pc_max_21days,time_spent_sum_30days,time_spent_max_30days,num_title_ep_sum_30days,num_title_ep_max_30days,num_title_ru_sum_30days,num_title_ru_max_30days,num_country_max_30days,num_city_max_30days,other_content_sum_30days,top_1_sum_30days,top_2_sum_30days,top_3_sum_30days,top_4_sum_30days,top_5_sum_30days,top_6_sum_30days,top_7_sum_30days,top_8_sum_30days,top_9_sum_30days,top_10_sum_30days,android_max_30days,smarttv_max_30days,iphone_max_30days,ipad_max_30days,apple_max_30days,pc_max_30days,activity_1to3,activity_1to7,activity_1to14,activity_7to14,activity_7to21,activity_7to30,activity_14to30,label
0,2019-04-02 00:00:00,11,8ba752f2c5,2019-02-10 00:00:00,2019-03-13 09:36:59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,59073.0,25595.0,10.0,4.0,10.0,4.0,1.0,20.0,6.0,17.0,20.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0
1,2019-04-02 00:00:00,10,752a6d96f7,2018-10-28 00:00:00,2019-04-01 20:51:29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,60.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,60.0,60.0,...,0.0,0.0,0.0,11.0,0.0,0.0,1.0,19801.0,10152.0,22.0,9.0,10.0,3.0,1.0,11.0,1.0,2.0,10.0,9.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,4.0,0.0,0.0,0.0,0.005807,0.005774,0.003030,0.521792,1.0
2,2019-04-02 00:00:00,4,827f6afef3,2018-12-14 00:00:00,2019-04-01 18:00:49,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,60.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,893.0,833.0,...,2.0,0.0,0.0,0.0,0.0,0.0,2.0,1493.0,833.0,12.0,5.0,11.0,5.0,1.0,6.0,3.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.937031,0.832238,0.598121,0.638308,0.0
3,2019-04-02 00:00:00,9,346e0f766c,2019-03-08 00:00:00,2019-03-31 23:05:17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,120.0,120.0,1.0,1.0,1.0,1.0,1.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,4656.0,2028.0,...,0.0,0.0,0.0,10.0,0.0,0.0,1.0,14724.0,4588.0,34.0,6.0,17.0,3.0,1.0,12.0,0.0,4.0,19.0,18.0,1.0,0.0,0.0,10.0,1.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,6.0,0.0,0.0,0.0,0.540452,0.536714,0.316218,0.585099,1.0
4,2019-04-02 00:00:00,10,0addbcc79a,2019-01-09 00:00:00,2019-03-25 17:41:09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,14.0,0.0,6.0,0.0,0.0,0.0,59165.0,13077.0,20.0,7.0,8.0,2.0,1.0,21.0,1.0,37.0,0.0,22.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,19.0,0.0,0.0,6.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.418660,0.0


In [0]:
df_test=pd.read_csv('/content/gdrive/My Drive/Churn_task/TEST_PREPARED.csv')

Проверим гипотезу, что выборка делилась на трейн и тест по email'ам. Посмотрим на пересечение множеств

In [9]:
len(set(df_test['email'].unique())&set(df['email'].unique()))

0

Теперь понятен принцип деления. Напишем функцию train_test_split, которая будет делить выборку

In [0]:
def train_test_split(df, pct=0.67): 
  unique_emails = set(df['email'].unique())
  train_emails = random.sample(unique_emails, int(pct * len(unique_emails)))
  x_train = df.query('email in @train_emails')
  y_train = x_train.pop('label')
  x_test = df.query('email not in @train_emails')
  y_test = x_test.pop('label')
  return x_train, y_train, x_test, y_test

Напишем функция препроцессинга данных, где создадим признаки последнего года, месяца и дня, а потом удалим все колонки ответственные за дату.

In [0]:
def date_preprocess(df):
  for name in df.filter(like='date').columns:
    df[name] = pd.to_datetime(df[name])
    df[name+'_weekday'] = df[name].dt.weekday.astype('int')
    df[name+'_year'] = df[name].dt.year.astype('int')
    df[name+'_month'] = df[name].dt.month.astype('int')
    df[name+'_day'] = df[name].dt.day.astype('int')
    df.drop(columns=name, inplace=True) 
  return df

In [0]:
def preprocess(df):
  df = date_preprocess(df)
  df=df.drop(columns ='email')
  return df

Обучим lightgbm на готовых данных, чтобы понимать в каком диапазоне должен быть скор

In [0]:
import lightgbm

In [0]:
x_train, y_train, x_test, y_test = train_test_split(df)

In [15]:
df.head()

,cut_date,days_to_end,email,first_date,last_date,num_country_max_1days,num_city_max_1days,android_max_1days,smarttv_max_1days,iphone_max_1days,ipad_max_1days,apple_max_1days,pc_max_1days,time_spent_sum_3days,time_spent_max_3days,num_title_ep_sum_3days,num_title_ep_max_3days,num_title_ru_sum_3days,num_title_ru_max_3days,num_country_max_3days,num_city_max_3days,other_content_sum_3days,top_1_sum_3days,top_2_sum_3days,top_3_sum_3days,top_4_sum_3days,top_5_sum_3days,top_6_sum_3days,top_7_sum_3days,top_8_sum_3days,top_9_sum_3days,top_10_sum_3days,android_max_3days,smarttv_max_3days,iphone_max_3days,ipad_max_3days,apple_max_3days,pc_max_3days,time_spent_sum_7days,time_spent_max_7days,...,top_10_sum_21days,android_max_21days,smarttv_max_21days,iphone_max_21days,ipad_max_21days,apple_max_21days,pc_max_21days,time_spent_sum_30days,time_spent_max_30days,num_title_ep_sum_30days,num_title_ep_max_30days,num_title_ru_sum_30days,num_title_ru_max_30days,num_country_max_30days,num_city_max_30days,other_content_sum_30days,top_1_sum_30days,top_2_sum_30days,top_3_sum_30days,top_4_sum_30days,top_5_sum_30days,top_6_sum_30days,top_7_sum_30days,top_8_sum_30days,top_9_sum_30days,top_10_sum_30days,android_max_30days,smarttv_max_30days,iphone_max_30days,ipad_max_30days,apple_max_30days,pc_max_30days,activity_1to3,activity_1to7,activity_1to14,activity_7to14,activity_7to21,activity_7to30,activity_14to30,label
0,2019-04-02 00:00:00,11,8ba752f2c5,2019-02-10 00:00:00,2019-03-13 09:36:59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,59073.0,25595.0,10.0,4.0,10.0,4.0,1.0,20.0,6.0,17.0,20.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0
1,2019-04-02 00:00:00,10,752a6d96f7,2018-10-28 00:00:00,2019-04-01 20:51:29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,60.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,60.0,60.0,...,0.0,0.0,0.0,11.0,0.0,0.0,1.0,19801.0,10152.0,22.0,9.0,10.0,3.0,1.0,11.0,1.0,2.0,10.0,9.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,4.0,0.0,0.0,0.0,0.005807,0.005774,0.003030,0.521792,1.0
2,2019-04-02 00:00:00,4,827f6afef3,2018-12-14 00:00:00,2019-04-01 18:00:49,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,60.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,893.0,833.0,...,2.0,0.0,0.0,0.0,0.0,0.0,2.0,1493.0,833.0,12.0,5.0,11.0,5.0,1.0,6.0,3.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.937031,0.832238,0.598121,0.638308,0.0
3,2019-04-02 00:00:00,9,346e0f766c,2019-03-08 00:00:00,2019-03-31 23:05:17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,120.0,120.0,1.0,1.0,1.0,1.0,1.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,4656.0,2028.0,...,0.0,0.0,0.0,10.0,0.0,0.0,1.0,14724.0,4588.0,34.0,6.0,17.0,3.0,1.0,12.0,0.0,4.0,19.0,18.0,1.0,0.0,0.0,10.0,1.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,6.0,0.0,0.0,0.0,0.540452,0.536714,0.316218,0.585099,1.0
4,2019-04-02 00:00:00,10,0addbcc79a,2019-01-09 00:00:00,2019-03-25 17:41:09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,14.0,0.0,6.0,0.0,0.0,0.0,59165.0,13077.0,20.0,7.0,8.0,2.0,1.0,21.0,1.0,37.0,0.0,22.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,19.0,0.0,0.0,6.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.418660,0.0


In [16]:
x_train = preprocess(x_train)
x_test = preprocess(x_test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [0]:
train_data = lightgbm.Dataset(x_train, label = y_train)
test_data = lightgbm.Dataset(x_test, label = y_test)

Обучим модель со случаными параметрами

In [0]:
param = {'num_leaves': 31, 'objective': 'binary'}
param['metric'] = 'auc'

In [19]:
model = lightgbm.train(param,train_data,
                       valid_sets=test_data,
                       num_boost_round=5000,
                       early_stopping_rounds=100)

[1]	valid_0's auc: 0.647078
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's auc: 0.649344
[3]	valid_0's auc: 0.659673
[4]	valid_0's auc: 0.661511
[5]	valid_0's auc: 0.666339
[6]	valid_0's auc: 0.668423
[7]	valid_0's auc: 0.669197
[8]	valid_0's auc: 0.670471
[9]	valid_0's auc: 0.670939
[10]	valid_0's auc: 0.671038
[11]	valid_0's auc: 0.672436
[12]	valid_0's auc: 0.673264
[13]	valid_0's auc: 0.673408
[14]	valid_0's auc: 0.673262
[15]	valid_0's auc: 0.672862
[16]	valid_0's auc: 0.672221
[17]	valid_0's auc: 0.671591
[18]	valid_0's auc: 0.670885
[19]	valid_0's auc: 0.671146
[20]	valid_0's auc: 0.67103
[21]	valid_0's auc: 0.671659
[22]	valid_0's auc: 0.67144
[23]	valid_0's auc: 0.671217
[24]	valid_0's auc: 0.67158
[25]	valid_0's auc: 0.67156
[26]	valid_0's auc: 0.670861
[27]	valid_0's auc: 0.671008
[28]	valid_0's auc: 0.671663
[29]	valid_0's auc: 0.671986
[30]	valid_0's auc: 0.672009
[31]	valid_0's auc: 0.671867
[32]	valid_0's auc: 0.670819
[33]	valid_0's auc: 0.

In [20]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_train,model.predict(x_train))

0.7200933015311248

Получили AUC-ROC=0.7674990397750523 

Улучшим его, добавив новые признаки. Используем TRAIN_ADDITIONAL_DATA и TRAIN_RAW_DATA

In [0]:
df_additional=pd.read_csv('/content/gdrive/My Drive/Churn_task/TRAIN_ADDITIONAL_DATA.csv')

In [0]:
df_new=pd.read_csv('/content/gdrive/My Drive/Churn_task/TRAIN_PREPARED.csv')

In [0]:
df_row=pd.read_csv('/content/gdrive/My Drive/Churn_task/TRAIN_RAW_DATA.csv')

In [24]:
df_additional.head()

,action,email,trial,calday
0,DELETE,9007366702,NaN,2018-10-26 03:26:41.607
1,CREATE,ad49ae7b1d,False,2018-12-01 18:16:56.971
2,CREATE,2165677f19,False,2018-12-01 18:16:58.380
3,CREATE,13ded413c2,False,2018-12-01 18:16:58.444
4,CREATE,032adbb404,False,2018-12-01 18:16:59.568


In [25]:
df_additional.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43494 entries, 0 to 43493
Data columns (total 4 columns):
action    43494 non-null object
email     43494 non-null object
trial     43295 non-null object
calday    43494 non-null object
dtypes: object(4)
memory usage: 1.3+ MB


Видим, что в признаке trial есть NaN, заменим их на 0 и приведем к типу int

In [0]:
df_additional['trial'] = df_additional['trial'].fillna(0).astype('int')

Приведем create к типу int 

In [0]:
df_additional['action']=(df_additional['action']=='CREATE').astype('int')

In [28]:
df_additional.head()

,action,email,trial,calday
0,0,9007366702,0,2018-10-26 03:26:41.607
1,1,ad49ae7b1d,0,2018-12-01 18:16:56.971
2,1,2165677f19,0,2018-12-01 18:16:58.380
3,1,13ded413c2,0,2018-12-01 18:16:58.444
4,1,032adbb404,0,2018-12-01 18:16:59.568


Добавим признаки last_act_last_day и last_trial_last_day, которые будут отвечать за последнюю активность (создание или удаление) и наличие триала соответственно.

In [0]:
df_additional['last_cal_day']=df_additional['email'].map(df_additional.groupby('email')['calday'].max().to_dict())

In [0]:
last_act_last_day=df_additional.sort_values(by='last_cal_day',ascending=False).drop_duplicates(subset='email').groupby('email')['action'].max().to_dict()

In [0]:
last_trial_last_day=df_additional.sort_values(by='last_cal_day',ascending=False).drop_duplicates(subset='email').groupby('email')['trial'].max().to_dict()

Добавим еще признаки trial_bin и action_bin, которые будут говорить о том был ли триал вообще и сколько различных действий предпринимал пользователь соответственно

In [0]:
trial_bin=df_additional.groupby('email')['trial'].max().to_dict()

In [0]:
action_bin=df_additional.groupby('email')['action'].count().to_dict()

Добавим эти признаки в датафрейм

In [0]:
df_new['trial_from_add'] = df_new['email'].map(last_trial_last_day)
df_new['action_from_add'] = df_new['email'].map(last_act_last_day)
df_new['trial_bin'] = df_new['email'].map(trial_bin)
df_new['action_bin'] = df_new['email'].map(action_bin)

Добавим признак, показывающий сколько дней оставалось до конца пробного периода в момент совершения последнего действия

In [0]:
dd=df_new.groupby('email')['days_to_end'].min().to_dict()

In [0]:
df_new['last_action_day_to_end']=df_new['email'].map(dd)

In [37]:
df_new.head()

,cut_date,days_to_end,email,first_date,last_date,num_country_max_1days,num_city_max_1days,android_max_1days,smarttv_max_1days,iphone_max_1days,ipad_max_1days,apple_max_1days,pc_max_1days,time_spent_sum_3days,time_spent_max_3days,num_title_ep_sum_3days,num_title_ep_max_3days,num_title_ru_sum_3days,num_title_ru_max_3days,num_country_max_3days,num_city_max_3days,other_content_sum_3days,top_1_sum_3days,top_2_sum_3days,top_3_sum_3days,top_4_sum_3days,top_5_sum_3days,top_6_sum_3days,top_7_sum_3days,top_8_sum_3days,top_9_sum_3days,top_10_sum_3days,android_max_3days,smarttv_max_3days,iphone_max_3days,ipad_max_3days,apple_max_3days,pc_max_3days,time_spent_sum_7days,time_spent_max_7days,...,apple_max_21days,pc_max_21days,time_spent_sum_30days,time_spent_max_30days,num_title_ep_sum_30days,num_title_ep_max_30days,num_title_ru_sum_30days,num_title_ru_max_30days,num_country_max_30days,num_city_max_30days,other_content_sum_30days,top_1_sum_30days,top_2_sum_30days,top_3_sum_30days,top_4_sum_30days,top_5_sum_30days,top_6_sum_30days,top_7_sum_30days,top_8_sum_30days,top_9_sum_30days,top_10_sum_30days,android_max_30days,smarttv_max_30days,iphone_max_30days,ipad_max_30days,apple_max_30days,pc_max_30days,activity_1to3,activity_1to7,activity_1to14,activity_7to14,activity_7to21,activity_7to30,activity_14to30,label,trial_from_add,action_from_add,trial_bin,action_bin,last_action_day_to_end
0,2019-04-02 00:00:00,11,8ba752f2c5,2019-02-10 00:00:00,2019-03-13 09:36:59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,59073.0,25595.0,10.0,4.0,10.0,4.0,1.0,20.0,6.0,17.0,20.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,1,1,1,1,2
1,2019-04-02 00:00:00,10,752a6d96f7,2018-10-28 00:00:00,2019-04-01 20:51:29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,60.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,60.0,60.0,...,0.0,1.0,19801.0,10152.0,22.0,9.0,10.0,3.0,1.0,11.0,1.0,2.0,10.0,9.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,4.0,0.0,0.0,0.0,0.005807,0.005774,0.003030,0.521792,1.0,0,0,0,7,2
2,2019-04-02 00:00:00,4,827f6afef3,2018-12-14 00:00:00,2019-04-01 18:00:49,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,60.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,893.0,833.0,...,0.0,2.0,1493.0,833.0,12.0,5.0,11.0,5.0,1.0,6.0,3.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.937031,0.832238,0.598121,0.638308,0.0,1,1,1,3,0
3,2019-04-02 00:00:00,9,346e0f766c,2019-03-08 00:00:00,2019-03-31 23:05:17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,120.0,120.0,1.0,1.0,1.0,1.0,1.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,4656.0,2028.0,...,0.0,1.0,14724.0,4588.0,34.0,6.0,17.0,3.0,1.0,12.0,0.0,4.0,19.0,18.0,1.0,0.0,0.0,10.0,1.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,6.0,0.0,0.0,0.0,0.540452,0.536714,0.316218,0.585099,1.0,0,0,1,2,0
4,2019-04-02 00:00:00,10,0addbcc79a,2019-01-09 00:00:00,2019-03-25 17:41:09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,59165.0,13077.0,20.0,7.0,8.0,2.0,1.0,21.0,1.0,37.0,0.0,22.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,19.0,0.0,0.0,6.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.418660,0.0,1,1,1,1,0


Добавим признак timelength = сколько дней пользовались сервисом со времени first_date и до last_date

In [0]:
df_new['timelength'] = pd.Series(pd.to_datetime(df_new['last_date'])-pd.to_datetime(df_new['first_date'])).apply(lambda x: x.days)

In [39]:
df_new.head()

,cut_date,days_to_end,email,first_date,last_date,num_country_max_1days,num_city_max_1days,android_max_1days,smarttv_max_1days,iphone_max_1days,ipad_max_1days,apple_max_1days,pc_max_1days,time_spent_sum_3days,time_spent_max_3days,num_title_ep_sum_3days,num_title_ep_max_3days,num_title_ru_sum_3days,num_title_ru_max_3days,num_country_max_3days,num_city_max_3days,other_content_sum_3days,top_1_sum_3days,top_2_sum_3days,top_3_sum_3days,top_4_sum_3days,top_5_sum_3days,top_6_sum_3days,top_7_sum_3days,top_8_sum_3days,top_9_sum_3days,top_10_sum_3days,android_max_3days,smarttv_max_3days,iphone_max_3days,ipad_max_3days,apple_max_3days,pc_max_3days,time_spent_sum_7days,time_spent_max_7days,...,pc_max_21days,time_spent_sum_30days,time_spent_max_30days,num_title_ep_sum_30days,num_title_ep_max_30days,num_title_ru_sum_30days,num_title_ru_max_30days,num_country_max_30days,num_city_max_30days,other_content_sum_30days,top_1_sum_30days,top_2_sum_30days,top_3_sum_30days,top_4_sum_30days,top_5_sum_30days,top_6_sum_30days,top_7_sum_30days,top_8_sum_30days,top_9_sum_30days,top_10_sum_30days,android_max_30days,smarttv_max_30days,iphone_max_30days,ipad_max_30days,apple_max_30days,pc_max_30days,activity_1to3,activity_1to7,activity_1to14,activity_7to14,activity_7to21,activity_7to30,activity_14to30,label,trial_from_add,action_from_add,trial_bin,action_bin,last_action_day_to_end,timelength
0,2019-04-02 00:00:00,11,8ba752f2c5,2019-02-10 00:00:00,2019-03-13 09:36:59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,59073.0,25595.0,10.0,4.0,10.0,4.0,1.0,20.0,6.0,17.0,20.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,1,1,1,1,2,31
1,2019-04-02 00:00:00,10,752a6d96f7,2018-10-28 00:00:00,2019-04-01 20:51:29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,60.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,60.0,60.0,...,1.0,19801.0,10152.0,22.0,9.0,10.0,3.0,1.0,11.0,1.0,2.0,10.0,9.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,4.0,0.0,0.0,0.0,0.005807,0.005774,0.003030,0.521792,1.0,0,0,0,7,2,155
2,2019-04-02 00:00:00,4,827f6afef3,2018-12-14 00:00:00,2019-04-01 18:00:49,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,60.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,893.0,833.0,...,2.0,1493.0,833.0,12.0,5.0,11.0,5.0,1.0,6.0,3.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.937031,0.832238,0.598121,0.638308,0.0,1,1,1,3,0,108
3,2019-04-02 00:00:00,9,346e0f766c,2019-03-08 00:00:00,2019-03-31 23:05:17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,120.0,120.0,1.0,1.0,1.0,1.0,1.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,4656.0,2028.0,...,1.0,14724.0,4588.0,34.0,6.0,17.0,3.0,1.0,12.0,0.0,4.0,19.0,18.0,1.0,0.0,0.0,10.0,1.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,6.0,0.0,0.0,0.0,0.540452,0.536714,0.316218,0.585099,1.0,0,0,1,2,0,23
4,2019-04-02 00:00:00,10,0addbcc79a,2019-01-09 00:00:00,2019-03-25 17:41:09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,59165.0,13077.0,20.0,7.0,8.0,2.0,1.0,21.0,1.0,37.0,0.0,22.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,19.0,0.0,0.0,6.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.418660,0.0,1,1,1,1,0,75


Воспользуемся теперь TRAIN_RAW_DATA 

av_tl -- признак, означающий суммарное время, которое пользователь провел на сервисе

In [0]:
av_tl=df_row.groupby('email')['timelength'].sum().to_dict()

num_titles -- признак, означающий количество уникальных тайтлов, просмотренных пользователем

In [0]:
num_titles=df_row.groupby('email')['title_ru'].nunique().to_dict()

In [0]:
df_new['av_tl']=df_new['email'].map(av_tl)
df_new['num_titles']=df_new['email'].map(num_titles)

Добавили все новые признаки в датафрейм, теперь посмотрим на качество

In [0]:
x_train_new, y_train_new, x_test_new, y_test_new = train_test_split(df_new)

In [44]:
x_train_new = preprocess(x_train_new)
x_test_new = preprocess(x_test_new)
train_data_new = lightgbm.Dataset(x_train_new, label = y_train_new)
test_data_new = lightgbm.Dataset(x_test_new, label = y_test_new)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

Потестим параметры для модели

In [0]:
parameters1 = {
    'application': 'binary',
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': 'true',
    'boosting': 'gbdt',
    'num_leaves': 69,
    'feature_fraction': 0.7,
    'bagging_fraction': 0.7,
    'bagging_freq': 1,
    'learning_rate': 0.1,
    
}

In [0]:
parameters2 = {
    'application': 'binary',
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': 'true',
    'boosting': 'gbdt',
    'num_leaves': 31,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 20,
    'learning_rate': 0.05,
    'verbose': 0
}

In [58]:
model1 = lightgbm.train(parameters1,
                       train_data_new,
                       valid_sets=test_data_new,
                       num_boost_round=5000,
                       early_stopping_rounds=100)

[1]	valid_0's auc: 0.829897
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's auc: 0.842985
[3]	valid_0's auc: 0.849172
[4]	valid_0's auc: 0.849537
[5]	valid_0's auc: 0.85001
[6]	valid_0's auc: 0.850824
[7]	valid_0's auc: 0.85337
[8]	valid_0's auc: 0.85518
[9]	valid_0's auc: 0.855852
[10]	valid_0's auc: 0.856182
[11]	valid_0's auc: 0.857131
[12]	valid_0's auc: 0.857961
[13]	valid_0's auc: 0.85823
[14]	valid_0's auc: 0.858407
[15]	valid_0's auc: 0.858724
[16]	valid_0's auc: 0.858298
[17]	valid_0's auc: 0.859907
[18]	valid_0's auc: 0.860067
[19]	valid_0's auc: 0.860389
[20]	valid_0's auc: 0.861437
[21]	valid_0's auc: 0.862785
[22]	valid_0's auc: 0.863471
[23]	valid_0's auc: 0.864596
[24]	valid_0's auc: 0.864472
[25]	valid_0's auc: 0.864615
[26]	valid_0's auc: 0.864574
[27]	valid_0's auc: 0.865005
[28]	valid_0's auc: 0.865385
[29]	valid_0's auc: 0.865804
[30]	valid_0's auc: 0.8657
[31]	valid_0's auc: 0.865625
[32]	valid_0's auc: 0.865803
[33]	valid_0's auc: 0.86

In [59]:
roc_auc_score(y_test_new,model1.predict(x_test_new))

0.8670175697509936

In [60]:
model2 = lightgbm.train(parameters2,
                       train_data_new,
                       valid_sets=test_data_new,
                       num_boost_round=5000,
                       early_stopping_rounds=100)

[1]	valid_0's auc: 0.839996
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's auc: 0.845441
[3]	valid_0's auc: 0.846139
[4]	valid_0's auc: 0.840114
[5]	valid_0's auc: 0.831564
[6]	valid_0's auc: 0.836813
[7]	valid_0's auc: 0.830763
[8]	valid_0's auc: 0.837641
[9]	valid_0's auc: 0.842471
[10]	valid_0's auc: 0.840977
[11]	valid_0's auc: 0.843801
[12]	valid_0's auc: 0.842184
[13]	valid_0's auc: 0.845672
[14]	valid_0's auc: 0.847294
[15]	valid_0's auc: 0.84652
[16]	valid_0's auc: 0.844953
[17]	valid_0's auc: 0.846311
[18]	valid_0's auc: 0.848742
[19]	valid_0's auc: 0.848256
[20]	valid_0's auc: 0.850332
[21]	valid_0's auc: 0.852746
[22]	valid_0's auc: 0.851975
[23]	valid_0's auc: 0.851213
[24]	valid_0's auc: 0.853121
[25]	valid_0's auc: 0.852322
[26]	valid_0's auc: 0.852095
[27]	valid_0's auc: 0.853224
[28]	valid_0's auc: 0.853159
[29]	valid_0's auc: 0.85303
[30]	valid_0's auc: 0.853815
[31]	valid_0's auc: 0.854799
[32]	valid_0's auc: 0.854392
[33]	valid_0's auc: 

In [61]:
roc_auc_score(y_test_new,model2.predict(x_test_new))

0.8672725968578932

Видно, что для обоих наборов параметров скор значительно улучшился относительно первоначального. При этом второй набор параметров для результат немного лучше.

Теперь обернем в функцию все манипуляции с данными, которые были проведены

In [0]:
def ultimate_preprocess(df_test,df_test_add,df_test_raw):
  df_additional = df_test_add
  df_new = df_test
  df_row = df_test_raw
  df_additional['trial'] = df_additional['trial'].fillna(0).astype('int')
  df_additional['action'] = (df_additional['action']=='CREATE').astype('int')
  df_additional['last_cal_day']=df_additional['email'].map(df_additional.groupby('email')['calday'].max().to_dict())
  last_act_last_day = df_additional.sort_values(by='last_cal_day',ascending=False).drop_duplicates(subset='email').groupby('email')['action'].max().to_dict()
  last_trial_last_day = df_additional.sort_values(by='last_cal_day',ascending=False).drop_duplicates(subset='email').groupby('email')['trial'].max().to_dict()
  trial_bin=df_additional.groupby('email')['trial'].max().to_dict()
  action_bin=df_additional.groupby('email')['action'].count().to_dict()
  df_new['trial_from_add'] = df_new['email'].map(last_trial_last_day)
  df_new['action_from_add'] = df_new['email'].map(last_act_last_day)
  df_new['trial_bin'] = df_new['email'].map(trial_bin)
  df_new['action_bin'] = df_new['email'].map(action_bin)
  dd=df_new.groupby('email')['days_to_end'].min().to_dict()
  df_new['last_action_day_to_end']=df_new['email'].map(dd)
  df_new['timelength'] = pd.Series(pd.to_datetime(df_new['last_date'])-pd.to_datetime(df_new['first_date'])).apply(lambda x: x.days)
  av_tl=df_row.groupby('email')['timelength'].sum().to_dict()
  num_titles=df_row.groupby('email')['title_ru'].nunique().to_dict()
  df_new['av_tl']=df_new['email'].map(av_tl)
  df_new['num_titles']=df_new['email'].map(num_titles)
  df_new = preprocess(df_new)
  return df_new

Сделаем финальное предсказание для тестовой выборки

In [0]:
test_prep = pd.read_csv('/content/gdrive/My Drive/Churn_task/TEST_PREPARED.csv')
test_add = pd.read_csv('/content/gdrive/My Drive/Churn_task/TEST_ADDITIONAL_DATA.csv')
test_raw = pd.read_csv('/content/gdrive/My Drive/Churn_task/TEST_RAW_DATA.csv')

In [0]:
df_ultimate = ultimate_preprocess(test_prep, test_add, test_raw)

In [0]:
submission = pd.DataFrame([])
submission['email'] = df_test['email']
submission['prediction'] = model2.predict(df_ultimate)

In [0]:
submission.to_csv('submission.csv')

In [0]:
from google.colab import files
files.download('submission.csv') 

Итоговый файл с предсказанием будет в submission_final.csv

---

